In [ ]:
import requests, re, time
import torch, torchvision
from torch import nn, optim
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
# For HTML parsing
from bs4 import BeautifulSoup as BS
# For making csv files to store prices of paintings
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Selenium stuff https://stackoverflow.com/questions/51046454/how-can-we-use-selenium-webdriver-in-colab-research-google-com
# !pip install selenium
# !apt-get update # to update ubuntu to correctly run apt install
# !apt install chromium-chromedriver
# !cp /usr/lib/chromium-browser/chromedriver /usr/bin
# import sys
# sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
# from selenium import webdriver

In [ ]:
# chrome_options = webdriver.ChromeOptions()
# chrome_options.add_argument('--headless')
# chrome_options.add_argument('--no-sandbox')
# chrome_options.add_argument('--disable-dev-shm-usage')
# wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

In [ ]:
# wd.get("https://www.artsy.net/collect?additional_gene_ids%5B0%5D=painting&price_range=1-20000")

In [ ]:
resp = requests.get("https://www.artsy.net/collect?additional_gene_ids%5B0%5D=painting&price_range=1-20000&at_auction=false&acquireable=true")
soup = BS(resp.content.decode())

In [ ]:
#PARSES 30 images found on the first page (TESTING)
count = 0
imgs = []
for tag in soup.find_all('a', class_='GridItem__Link-l61twt-1 krUChM'):
  if count < 30:
      imgs.append(tag.img.get('src'))
      count+=1

print(len(imgs))
imgs[29]

In [ ]:
#PARSES PRICES (TESTING)
prices = []
count = 0
for tag in soup.find_all('div', class_='Box-sc-15se88d-0 hXGxSA'):
  if count < 30:
    prices.append(tag.div.contents[3].div.div.get_text())
    count+=1

for i in range(30):
  print(imgs[i],prices[i])

In [ ]:
#PARSING MULTIPLE PAGES
prices = []
imgs = []
for page in range(1, 101):
  resp = requests.get(f'https://www.artsy.net/collect?page={page}&additional_gene_ids%5B0%5D=painting&price_range=1-20000&at_auction=false&acquireable=true')
  soup = BS(resp.content.decode())
  count_imgs = 0
  count_prices = 0
  #Get imgs on page
  for tag in soup.find_all('a', class_='GridItem__Link-l61twt-1 krUChM'):
    if count_imgs < 30:
        imgs.append(tag.img.get('src'))
        count_imgs+=1
  #Get prices on page
  for tag in soup.find_all('div', class_='Box-sc-15se88d-0 hXGxSA'):
    if count_prices < 30:
      prices.append(tag.div.contents[3].div.div.get_text())
      count_prices+=1

for i in range(len(imgs)):
  print(imgs[i],prices[i])

https://d7hftxdivxxvm.cloudfront.net/?resize_to=fit&width=400&height=396&quality=80&src=https%3A%2F%2Fd32dm0rphc51dk.cloudfront.net%2FNvN0sERNUsxHFASQu7evDg%2Flarge.jpg $12,000 
https://d7hftxdivxxvm.cloudfront.net/?resize_to=fit&width=400&height=322&quality=80&src=https%3A%2F%2Fd32dm0rphc51dk.cloudfront.net%2FDhd7ZcjiC0EnK1XOsvyBMw%2Flarge.jpg $6,250 
https://d7hftxdivxxvm.cloudfront.net/?resize_to=fit&width=400&height=421&quality=80&src=https%3A%2F%2Fd32dm0rphc51dk.cloudfront.net%2FhYDJ73zH6YE9Xj5-BfbX9g%2Flarge.jpg $5,000 
https://d7hftxdivxxvm.cloudfront.net/?resize_to=fit&width=400&height=400&quality=80&src=https%3A%2F%2Fd32dm0rphc51dk.cloudfront.net%2FXcauCCXIV-01hddac-2axA%2Flarge.jpg $475 
https://d7hftxdivxxvm.cloudfront.net/?resize_to=fit&width=400&height=373&quality=80&src=https%3A%2F%2Fd32dm0rphc51dk.cloudfront.net%2FY-eLzbdEC4IyLLLzXHgzqg%2Flarge.jpg $5,100 
https://d7hftxdivxxvm.cloudfront.net/?resize_to=fit&width=400&height=268&quality=80&src=https%3A%2F%2Fd32dm0rphc51dk

In [ ]:
print(len(prices),len(imgs))

3000 3000


In [ ]:
#CLEAN UP CURRENCIES
!pip install forex-python

In [ ]:
#CLEAN UP CURRENCIES
from forex_python.converter import CurrencyRates
c = CurrencyRates()
P_2_USD = c.get_rate('GBP', 'USD') # convert pounds to usd
E_2_USD = c.get_rate('EUR', 'USD') # convert euros to usd
i = 0;
for price in prices:
  if price[0] == '€':
    old_val = float("".join(d for d in price if d.isdigit()))
    usd_p = old_val * E_2_USD
    new_p = '${:,.2f}'.format(usd_p)
    prices[i] = new_p
  elif price[0] == '£':
    old_val = float("".join(d for d in price if d.isdigit()))
    usd_p = old_val * P_2_USD
    new_p = '${:,.2f}'.format(usd_p)
    prices[i] = new_p
  i+=1

In [ ]:
P_2_USD

1.3787194084

In [ ]:
df = pd.DataFrame()
df.insert(loc=0, column = 'price', value = prices)
df.insert(loc=0, column = 'image', value = imgs)


In [ ]:
count = 0
for price in prices:
  if price[0] == '£':
    print(price[0])
    count +=1

In [ ]:
df


,image,price
0,https://d7hftxdivxxvm.cloudfront.net/?resize_t...,"$12,000"
1,https://d7hftxdivxxvm.cloudfront.net/?resize_t...,"$6,250"
2,https://d7hftxdivxxvm.cloudfront.net/?resize_t...,"$5,000"
3,https://d7hftxdivxxvm.cloudfront.net/?resize_t...,$475
4,https://d7hftxdivxxvm.cloudfront.net/?resize_t...,"$5,100"
...,...,...
2995,https://d7hftxdivxxvm.cloudfront.net/?resize_t...,"$7,282.52"
2996,https://d7hftxdivxxvm.cloudfront.net/?resize_t...,$450
2997,https://d7hftxdivxxvm.cloudfront.net/?resize_t...,"$9,895"
2998,https://d7hftxdivxxvm.cloudfront.net/?resize_t...,"$8,000"


In [ ]:
df.to_csv('/content/drive/My Drive/CPEN291 Project/Artsy_Data.csv')